## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-07-22-08-31-36 +0000,world,Detention of two Italians at ‘Alligator Alcatr...,https://www.washingtonpost.com/world/2025/07/2...
1,2025-07-22-05-00-00 +0000,world,"The Taliban banned beauty salons, leaving wome...",https://www.washingtonpost.com/world/2025/07/2...
2,2025-07-22-01-15-18 +0000,world,"U.N. says facilities hit, guesthouse in Gaza r...",https://www.washingtonpost.com/world/2025/07/2...
3,2025-07-22-00-10-57 +0000,world,The for-profit companies behind Israeli-U.S. n...,https://www.washingtonpost.com/world/2025/07/2...
4,2025-07-21-20-51-17 +0000,world,"Bangladesh air force jet crashes into school, ...",https://www.washingtonpost.com/world/2025/07/2...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2304/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
18,gaza,2
20,israeli,2
0,detention,1
1,two,1
4,alcatraz,1


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
3,2025-07-22-00-10-57 +0000,world,The for-profit companies behind Israeli-U.S. n...,https://www.washingtonpost.com/world/2025/07/2...,10
2,2025-07-22-01-15-18 +0000,world,"U.N. says facilities hit, guesthouse in Gaza r...",https://www.washingtonpost.com/world/2025/07/2...,9
6,2025-07-21-11-52-58 +0000,world,Ecuador’s most notorious drug lord taken to U....,https://www.washingtonpost.com/world/2025/07/2...,9
1,2025-07-22-05-00-00 +0000,world,"The Taliban banned beauty salons, leaving wome...",https://www.washingtonpost.com/world/2025/07/2...,8
4,2025-07-21-20-51-17 +0000,world,"Bangladesh air force jet crashes into school, ...",https://www.washingtonpost.com/world/2025/07/2...,8


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
3,10,2025-07-22-00-10-57 +0000,world,The for-profit companies behind Israeli-U.S. n...,https://www.washingtonpost.com/world/2025/07/2...
6,9,2025-07-21-11-52-58 +0000,world,Ecuador’s most notorious drug lord taken to U....,https://www.washingtonpost.com/world/2025/07/2...
1,8,2025-07-22-05-00-00 +0000,world,"The Taliban banned beauty salons, leaving wome...",https://www.washingtonpost.com/world/2025/07/2...
4,8,2025-07-21-20-51-17 +0000,world,"Bangladesh air force jet crashes into school, ...",https://www.washingtonpost.com/world/2025/07/2...
0,7,2025-07-22-08-31-36 +0000,world,Detention of two Italians at ‘Alligator Alcatr...,https://www.washingtonpost.com/world/2025/07/2...
5,7,2025-07-21-17-55-07 +0000,world,Houthi attacks take toll on Israel’s Red Sea port,https://www.washingtonpost.com/world/2025/07/2...
7,7,2025-07-21-10-03-35 +0000,world,China says Wells Fargo executive ‘involved in ...,https://www.washingtonpost.com/world/2025/07/2...
8,7,2025-07-21-10-00-20 +0000,world,Trump’s tariffs could upend U.S. trade with E....,https://www.washingtonpost.com/world/2025/07/2...
9,6,2025-07-21-10-00-01 +0000,world,Saving Istanbul’s ancient structures from the ...,https://www.washingtonpost.com/world/interacti...
2,5,2025-07-22-01-15-18 +0000,world,"U.N. says facilities hit, guesthouse in Gaza r...",https://www.washingtonpost.com/world/2025/07/2...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
